In [1]:
# CÉLULA 1 - MONTAGEM DO GOOGLE DRIVE E DEFINIÇÃO DE CAMINHOS GLOBAIS
from google.colab import drive #importa biblioteca para conectar Colab com Drive (onde o database está)
drive.mount('/content/drive') #acessa meu Drive como uma pasta local

# Defina os caminhos para suas pastas e arquivos no Google Drive
  #Armazena na variável o caminho do database
DATA_PATH = "/content/drive/MyDrive/RAG/databaseRag/all"
  #Armazena na variável o caminho do model LLM (que está no drive)
LLAMA_MODEL_PATH = "/content/drive/MyDrive/RAG/models/new/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf"
  #Armazena na variável o caminho onde o banco ChromaDB será salvo. O Chroma vai guardar os vetores numéricos que eram os arigos
CHROMA_PERSIST_DIR = "/content/drive/MyDrive/RAG/databaseRag/chroma_db_new"

#Printar na tela os valores das variáveis (útil para verificação, se foi linkado com sucesso)
print(f"Caminho dos Dados: {DATA_PATH}")
print(f"Caminho do Modelo LLM: {LLAMA_MODEL_PATH}")
print(f"Caminho do Banco de Dados Chroma: {CHROMA_PERSIST_DIR}")

Mounted at /content/drive
Caminho dos Dados: /content/drive/MyDrive/RAG/databaseRag/all
Caminho do Modelo LLM: /content/drive/MyDrive/RAG/models/new/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf
Caminho do Banco de Dados Chroma: /content/drive/MyDrive/RAG/databaseRag/chroma_db_new


In [2]:
#CÉLULA 2 - REMOÇÃO DO ANTIGO BD

#importa biblioteca para interagir com o sistema operacional
import os
#importa a biblioteca para fazer operações de alto nível em arquivos, como apagar um diretório inteiro
import shutil


#verifica se a pasta do banco de dados ChromaDB (definida na céula 1) já existe
if os.path.exists(CHROMA_PERSIST_DIR): #caso exista
    print(f"Removendo o banco de dados antigo em: {CHROMA_PERSIST_DIR}") #informa que será excluida
    shutil.rmtree(CHROMA_PERSIST_DIR) #apaga a pasta e o conteúdo que houver dentro dela
    print("Banco de dados antigo removido com sucesso.")
else: #caso não exista, apenas informa:
    print("Nenhum banco de dados antigo encontrado para remover.")

Nenhum banco de dados antigo encontrado para remover.


In [3]:
# CÉLULA 3 - INSTALAÇÃO PARA CPU
print("⬇️ 1. Instalando bibliotecas base...")
!pip install -q pdfminer.six pypdf docx2txt sentence-transformers gradio

print("⬇️ 2. Instalando Banco de Dados e LangChain...")
!pip install -q chromadb==0.4.22
!pip install -q langchain==0.1.0 langchain-community==0.0.10 langchain-core==0.1.8 --no-deps

print("⬇️ 3. Instalando Llama-cpp (Versão Nativa CPU)...")
# Desinstalamos qualquer versão prévia e instalamos a padrão (sem utilizar acelerador CUDA)
!pip uninstall -y llama-cpp-python
!pip install llama-cpp-python --no-cache-dir

print("\n✅ Ambiente pronto para uso em CPU!")

⬇️ 1. Instalando bibliotecas base...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 21.0 MB/s eta 0:00:00
⬇️ 2. Instalando Banco de Dados e LangChain...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.2 MB/s 

In [4]:
# CÉLULA 4 - DEMAIS IMPORTAÇÕES


#biblioteca para interagir com arquivos e pastas do sistema
import os
import shutil
#importa divisor de texto, que quebra documentos longos em chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
#importa a classe do ChromaDB, onde os chunks serão armazenados e indexados
from langchain_community.vectorstores import Chroma
#importa um fluxo pré construido que conecta a busca de documentos com geração de respostas pelo LLM
from langchain.chains import RetrievalQA
#importa o conector que permite ao LangChain usar o modelo Llama 3 do arquivo .guff
from langchain_community.llms import LlamaCpp
#importa ferramenta que cria templates de prompt, estrtuturando a pergunta enviada ao LLM
from langchain.prompts import PromptTemplate
#importa função para limpar metadados complexos
from langchain_community.vectorstores.utils import filter_complex_metadata
#carregador de arquivos unstructured, possibilita a leitura de arquivos com diretos formatos **ponto chave p ler artigos cientificos e revistas
from langchain_community.document_loaders import UnstructuredFileLoader
#importa 'callback' para exibir a resposta do LLM em tempo real ~vai printando cfme encontra resposta~
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
#importa classe principal que carrega modelo para transformar trechos de texto em vetores numéricos (embeddings)
from langchain_community.embeddings import SentenceTransformerEmbeddings


In [5]:
# CÉLULA 5 - FUNÇÕES DE CARREGAMENTO E CHUNKING (OTIMIZADA CPU)
import os
from langchain_community.document_loaders import PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_documents_from_folder(folder_path):
    documents = []
    # Filtra apenas PDFs e ignora arquivos temporários ~$
    files = [f for f in os.listdir(folder_path) if f.endswith('.pdf') and not f.startswith("~$")]

    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
            # PDFMiner é CPU-bound (usa apenas o processador)
            print(f"📄 Extraindo texto (CPU): {file}...")
            loader = PDFMinerLoader(file_path)
            documents.extend(loader.load())
        except Exception as e:
            print(f"❌ Erro ao processar {file}: {e}")

    print(f"\n✅ Carregamento finalizado. Elementos: {len(documents)}")
    return documents

def split_documents_into_chunks(documents, chunk_size=800, chunk_overlap=200):
    # O split de texto é uma operação leve de CPU
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"🧩 {len(chunks)} chunks gerados com sucesso.")
    return chunks

In [ ]:
# CÉLULA 6 - CRIAÇÃO DO BANCO
import os
import glob
import numpy as np

# Patch para NumPy 2.x
if not hasattr(np, 'float_'): np.float_ = np.float64

from langchain_community.document_loaders import PDFMinerLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

def create_vector_db():
    # 1. Caminho dos arquivos
    DATA_PATH = "/content/drive/MyDrive/RAG/databaseRag/artigos"

    print(f"🚀 Iniciando leitura com PDFMiner (Análise de Layout)...")
    pdf_files = glob.glob(os.path.join(DATA_PATH, "*.pdf"))

    all_documents = []
    for pdf_file in pdf_files:
        try:
            print(f"   - Lendo colunas e tabelas: {os.path.basename(pdf_file)}...")
            loader = PDFMinerLoader(pdf_file)
            all_documents.extend(loader.load())
        except Exception as e:
            print(f"   ❌ Erro no PDFMiner: {e}")

    # 2. Chunking
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=250)
    chunks = text_splitter.split_documents(all_documents)
    print(f"🧩 {len(chunks)} chunks gerados com inteligência de layout.")

    # 3. Criação do Banco
    embedding_function = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_function,
        persist_directory=CHROMA_PERSIST_DIR
    )
    vector_db.persist()
    print("✅ SUCESSO! Banco de dados criado.")
    return vector_db

vector_db = create_vector_db()

🚀 Iniciando leitura com PDFMiner (Análise de Layout)...
   - Lendo colunas e tabelas: 5371-Texto do artigo-16016-1-10-20081007.pdf...


   - Lendo colunas e tabelas: 4284-28572-1-PB.pdf...
   - Lendo colunas e tabelas: IIDD225 PDF.pdf...
   - Lendo colunas e tabelas: rgenfermagem,+551-556.pdf...
   - Lendo colunas e tabelas: 35284-Texto do Artigo-253541-1-10-20241114.pdf...
   - Lendo colunas e tabelas: ijrb-17-883.pdf...
   - Lendo colunas e tabelas: ali,+2+CUIDADOS+DE+ENFERMAGEM+A+PACIENTES+EM+PRÉ-OPERATÓRIO+PROPOSTA+DE+CHECKLIST.pdf...


In [ ]:
# CÉLULA FINAL - CHATBOT RAG (VERSÃO CPU)
import os
import ctypes
import glob
import gradio as gr
import numpy as np

# Patch para evitar erro de drivers ausentes
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu:/usr/local/lib'
if not hasattr(np, 'float_'): np.float_ = np.float64

from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

print("🤖 Iniciando CardioBot (MODO CPU)...")

# 1. Caminhos
CHROMA_PATH = "/content/db_cardio_local" # Certifique-se de que a Célula 8 criou este banco
base_path = "/content/drive/MyDrive/RAG"
model_files = glob.glob(os.path.join(base_path, "**/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf"), recursive=True)
model_path = model_files[0]

# 2. Modelo
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=0,      # Força CPU
    n_batch=512,
    n_ctx=2048,
    temperature=0.1,
    verbose=False
)

# 3. Banco de Dados
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# 4. Prompt e Chain
template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Você é um profissional de saúde, especialista em orientação pré e pós-operatória de cirurgia cardíaca.
Forneça respostas padronizadas, seguras, objetivas e em linguagem acessível ao paciente. Suas orientações são estritamente baseadas no contexto fornecido.

Regra de Segurança: Se a informação for individualizada, ou não estiver no contexto, a resposta deve ser:
"Esta é uma questão específica do seu histórico médico e deve ser discutida diretamente com seu médico cardiologista ou a equipe de enfermagem."
OU "Essa informação depende de uma avaliação individualizada. Por favor, consulte seu médico ou a equipe de saúde responsável.

5. Responda sempre em Português do Brasil.<|eot_id|><|start_header_id|>user<|end_header_id|>
CONTEXTO:
{context}

PERGUNTA:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": PromptTemplate.from_template(template)}
)

# 5. Chat
def responder(pergunta, historico):
    if not pergunta: return ""
    res = qa_chain.invoke({"query": pergunta})
    return res["result"].split("assistant")[-1].replace("<|eot_id|>", "").strip()

demo = gr.ChatInterface(fn=responder, title="🫀 CardioBot (CPU Mode)")
demo.launch(share=True, debug=True)